In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import time

In [2]:
start_time = time.time()


df = pd.read_parquet('../time_seires/year/dfTime_TLC-2022.parquet', engine='pyarrow')
print(df.shape)

end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

(5100018, 9)
執行時間: 0.7761569023132324 秒


In [3]:
df.head()

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2022,1,1,0,5,3,True,16
1,lyft,2022,1,1,0,5,4,True,49
2,lyft,2022,1,1,0,5,5,True,1
3,lyft,2022,1,1,0,5,6,True,1
4,lyft,2022,1,1,0,5,7,True,115


In [3]:
test = pd.read_parquet('../time_seires/month/dfTime_TLC-2023-1.parquet', engine='pyarrow')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430226 entries, 0 to 430225
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Name          430226 non-null  object
 1   year          430226 non-null  int32 
 2   month         430226 non-null  int32 
 3   day           430226 non-null  int32 
 4   hour          430226 non-null  int32 
 5   weekday       430226 non-null  int32 
 6   PULocationID  430226 non-null  int64 
 7   is_holiday    430226 non-null  bool  
 8   次數            430226 non-null  int64 
dtypes: bool(1), int32(5), int64(2), object(1)
memory usage: 18.5+ MB


In [4]:
test = pd.get_dummies(test, columns=['Name'], prefix='Name')
test = test.drop('year', axis=1)
test = test.astype({'is_holiday': 'int', 
                'Name_lyft': 'int', 
                'Name_uber': 'int', 
                'Name_yellow': 'int'})
test = test.rename(columns={'次數': 'count'})
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430226 entries, 0 to 430225
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   month         430226 non-null  int32
 1   day           430226 non-null  int32
 2   hour          430226 non-null  int32
 3   weekday       430226 non-null  int32
 4   PULocationID  430226 non-null  int64
 5   is_holiday    430226 non-null  int32
 6   count         430226 non-null  int64
 7   Name_lyft     430226 non-null  int32
 8   Name_uber     430226 non-null  int32
 9   Name_yellow   430226 non-null  int32
dtypes: int32(8), int64(2)
memory usage: 19.7 MB


In [5]:
X_actual = test.drop('count', axis=1)
y_actual = test['count'].values
print(X_actual.shape)
print(y_actual.shape)

(430226, 9)
(430226,)


# Keras LSTM

In [5]:
# from sklearn.preprocessing import MinMaxScaler

# training_set = uber['次數'].values.reshape(-1, 1)


# scaler = MinMaxScaler()
# training_set_scaled = scaler.fit_transform(training_set)

In [6]:
# X_train = []   #預測點的前 60 天的資料
# y_train = []   #預測點
# for i in range(48, 7000):  # 1258 是訓練集總數
#     X_train.append(training_set_scaled[i-48:i, 0])
#     y_train.append(training_set_scaled[i, 0])
# X_train, y_train = np.array(X_train), np.array(y_train)

In [7]:
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [8]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.layers import Dropout

# regressor = Sequential()

In [9]:
# regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50, return_sequences=True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50, return_sequences=True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50))
# regressor.add(Dropout(0.2))

# regressor.add(Dense(units=1))

In [10]:
# regressor.compile(optimizer='adam', loss='mean_squared_error')

# 進行訓練
# history = regressor.fit(X_train, y_train, epochs=50, batch_size=32)

## scikit-learn

In [6]:
df = pd.get_dummies(df, columns=['Name'], prefix='Name')

In [7]:
# from sklearn.preprocessing import LabelEncoder
# label = LabelEncoder()
# df['Name'] = label.fit_transform(df['Name'])

# df['Name'].value_counts()

In [8]:
df = df.drop('year', axis=1)
df = df.astype({'is_holiday': 'int', 
                'Name_lyft': 'int', 
                'Name_uber': 'int', 
                'Name_yellow': 'int'})
df = df.rename(columns={'次數': 'count'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100018 entries, 0 to 5100017
Data columns (total 10 columns):
 #   Column        Dtype
---  ------        -----
 0   month         int32
 1   day           int32
 2   hour          int32
 3   weekday       int32
 4   PULocationID  int64
 5   is_holiday    int32
 6   count         int64
 7   Name_lyft     int32
 8   Name_uber     int32
 9   Name_yellow   int32
dtypes: int32(8), int64(2)
memory usage: 233.5 MB


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor



X = df.drop('count', axis=1)
y = df['count']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.fit_transform(X_valid)


# models = {'KNN': KNeighborsRegressor(),
#           'SVR': SVR(),
#           'Decision Tree': DecisionTreeRegressor(), 
#           'RandomForest': RandomForestRegressor(),
#           'XGBoost': XGBRegressor(n_estimators=500, 
#                                   learning_rate=0.05, 
#                                   n_jobs=-1, 
#                                   random_state=0),
#           'LightGBM': LGBMRegressor(n_estimators=500, 
#                                     learning_rate=0.1, 
#                                     n_jobs=-1, 
#                                     random_state=0)}
# results = []

# for model in models.values():
#     kf = KFold(n_splits=6, random_state=42, shuffle=True)
#     cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kf)
#     results.append(cv_results)

# # plt.boxplot(results, labels=models.keys())
# sns.boxplot(results, x=models.keys())
# plt.show()

In [16]:
# model = RandomForestRegressor(random_state=1, n_jobs=-1)
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print(f'MAE: {mae}\nR2: {r2}')

## LightGBM

In [ ]:
# start_time = time.time()


# from sklearn.model_selection import GridSearchCV
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# param_grid = {'n_estimators': range(100, 501, 100),
#               'learning_rate': np.logspace(0, -2, 3), 
#               'max_depth': range(3, 20, 2)}

# lgbm = LGBMRegressor(n_jobs=-1, 
#                      random_state=0)

# model_cv = GridSearchCV(lgbm, param_grid, cv=kf)
# model_cv.fit(X_train_scaled, y_train)
# print(model_cv.best_params_, model_cv.best_score_)


# end_time = time.time()
# execution_time = end_time - start_time
# print("執行時間:", execution_time, "秒")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.063469 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 324
[LightGBM] [Info] Number of data points in the train set: 3264011, number of used features: 7
[LightGBM] [Info] Start training from score 49.191474


In [17]:
start_time = time.time()


model = LGBMRegressor(n_estimators=100, 
                      learning_rate=1, 
                      max_depth=8, 
                      num_leaves=31,
                      n_jobs=-1, 
                      random_state=0, 
                      verbose=-1)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_valid_scaled)

mae = mean_absolute_error(y_valid, y_pred)
r2 = r2_score(y_valid, y_pred)

print(f'MAE: {mae}\nR2: {r2}')


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

MAE: 15.107376763178433
R2: 0.868309928829479
執行時間: 6.936546564102173 秒


In [21]:
y_actual

array([ 1, 12, 49, ...,  5, 11, 41], dtype=int64)

In [23]:
prediction

array([68.23359422, 68.23359422, 68.23359422, ..., 56.91757214,
       56.91757214, 56.91757214])

In [18]:
prediction = model.predict(X_actual)
mean_absolute_error(y_actual, prediction)
r2 = r2_score(y_actual, prediction)
print(f'MAE: {mae}\nR2: {r2}')

MAE: 15.107376763178433
R2: -0.04944763525821494


MAE: 8.857072736240927
R2: -0.11156618127730145


## XGBoost

In [21]:
start_time = time.time()


from sklearn.model_selection import GridSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {'n_estimators': range(100, 501, 100),
              'learning_rate': np.logspace(0,-2, 3)}

xgbr = XGBRegressor(random_state=0)

model_cv = GridSearchCV(xgbr, param_grid, cv=kf)
model_cv.fit(X_train, y_train)

# 查看表現最好的超參數和平均交叉驗證分數
print(model_cv.best_params_, model_cv.best_score_)


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

KeyboardInterrupt: 

In [ ]:
start_time = time.time()


model = XGBRegressor(n_estimators=1000, 
                     learning_rate=0.1, 
                     n_jobs=-1, 
                     random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}\nR2: {r2}')


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

In [ ]:
from lazypredict.Supervised import LazyRegressor

LC = LazyRegressor(ignore_warnings=True, random_state=0, verbose=False)
models, predictions = LC.fit(X_train, X_test, y_train, y_test)
models

## NN

In [ ]:
# start_time = time.time()


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import LeakyReLU

# n_cols = X_train.shape[1]

# model = Sequential([Dense(14, input_shape=(n_cols,)),
#                     BatchNormalization(),
#                     LeakyReLU(),
#                     Dense(14, input_shape=(n_cols,)),
#                     BatchNormalization(),
#                     LeakyReLU(),
#                     Dense(1)
# ])

# model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
# model.summary()

# # 擬合X, y
# history = model.fit(X_train, y_train, validation_split=0.3, batch_size=100, epochs=10, verbose=2)


# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.legend(['training', 'validation'], loc='upper left')
# plt.show()


# result = model.evaluate(X_test, y_test)
# print(result)


# end_time = time.time()
# execution_time = end_time - start_time
# print("執行時間:", execution_time, "秒")